Load the modules

In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from src import datareader, enums

Load in the data and split between training and testing

In [2]:
filename = '/Users/mhogan/Documents/heartfailurepred/heart.csv'
keys, features, labels = datareader.get_csv_features_labels(filename)
tt_splits = train_test_split(features, labels, test_size=0.33)
features_train, features_test, labels_train, labels_test = tt_splits

Normalize the data

In [3]:
normalized_features_train = preprocessing.normalize(features_train.swapaxes(0, 1)).swapaxes(0, 1)
normalized_features_test = preprocessing.normalize(features_test.swapaxes(0, 1)).swapaxes(0, 1)
normalized_features_test = normalized_features_test.reshape((len(normalized_features_test), len(keys)-1))

Define the NN model

In [4]:
n_epochs = 1000

# Single hidden layer NN
heart_disease_model = tf.keras.Sequential([
    layers.Dense(len(keys) * 2),
    layers.Dense(1)
])

# Compile the model and fit
heart_disease_model.compile(loss=tf.losses.BinaryCrossentropy(),
                            optimizer=tf.optimizers.Adam())
heart_disease_model.fit(normalized_features_train, labels_train, epochs=n_epochs)

2021-10-10 14:02:07.232434: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-10 14:02:07.307575: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/1000
20/20 [==============================] - 1s 2ms/step - loss: 8.8247
Epoch 2/1000
20/20 [==============================] - 0s 2ms/step - loss: 7.1577
Epoch 3/1000
20/20 [==============================] - 0s 6ms/step - loss: 2.4376
Epoch 4/1000
20/20 [==============================] - 0s 7ms/step - loss: 1.4858
Epoch 5/1000
20/20 [==============================] - 0s 3ms/step - loss: 1.3330
Epoch 6/1000
20/20 [==============================] - 0s 5ms/step - loss: 1.2196
Epoch 7/1000
20/20 [==============================] - 0s 4ms/step - loss: 1.2096
Epoch 8/1000
20/20 [==============================] - 0s 3ms/step - loss: 1.1560
Epoch 9/1000
20/20 [==============================] - 0s 3ms/step - loss: 1.1268
Epoch 10/1000
20/20 [==============================] - 0s 4ms/step - loss: 1.0914
Epoch 11/1000
20/20 [==============================] - 0s 2ms/step - loss: 1.0484
Epoch 12/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.9985
Epoch 13/1000
20/20 [====

20/20 [==============================] - 0s 2ms/step - loss: 0.4959
Epoch 101/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4512
Epoch 102/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4624
Epoch 103/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4508
Epoch 104/1000
20/20 [==============================] - 0s 3ms/step - loss: 0.4476
Epoch 105/1000
20/20 [==============================] - 0s 7ms/step - loss: 0.4660
Epoch 106/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.4384
Epoch 107/1000
20/20 [==============================] - 0s 3ms/step - loss: 0.4469
Epoch 108/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4564
Epoch 109/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4281
Epoch 110/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4635
Epoch 111/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4773
Epoch 112/1000
20/2

20/20 [==============================] - 0s 2ms/step - loss: 0.3805
Epoch 199/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3750
Epoch 200/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4134
Epoch 201/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3494
Epoch 202/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3897
Epoch 203/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3927
Epoch 204/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3543
Epoch 205/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3420
Epoch 206/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3579
Epoch 207/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4173
Epoch 208/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3736
Epoch 209/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3874
Epoch 210/1000
20/2

20/20 [==============================] - 0s 2ms/step - loss: 0.5724
Epoch 297/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4661
Epoch 298/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4175
Epoch 299/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4786
Epoch 300/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4535
Epoch 301/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3871
Epoch 302/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4242
Epoch 303/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4241
Epoch 304/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3790
Epoch 305/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4078
Epoch 306/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4169
Epoch 307/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3936
Epoch 308/1000
20/2

...

20/20 [==============================] - 0s 2ms/step - loss: 0.4420
Epoch 591/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3577
Epoch 592/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3643
Epoch 593/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4086
Epoch 594/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4043
Epoch 595/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3911
Epoch 596/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3761
Epoch 597/1000
20/20 [==============================] - 0s 3ms/step - loss: 0.4278
Epoch 598/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4241
Epoch 599/1000
20/20 [==============================] - 0s 3ms/step - loss: 0.3885
Epoch 600/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4100
Epoch 601/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3630
Epoch 602/1000
20/2

20/20 [==============================] - 0s 2ms/step - loss: 0.3452
Epoch 689/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3858
Epoch 690/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3451
Epoch 691/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3414
Epoch 692/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3867
Epoch 693/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3600
Epoch 694/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4023
Epoch 695/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3866
Epoch 696/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3846
Epoch 697/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3613
Epoch 698/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3891
Epoch 699/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3669
Epoch 700/1000
20/2

20/20 [==============================] - 0s 2ms/step - loss: 0.3783
Epoch 787/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3339
Epoch 788/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3906
Epoch 789/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3488
Epoch 790/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3669
Epoch 791/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3552
Epoch 792/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4157
Epoch 793/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3660
Epoch 794/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3414
Epoch 795/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4357
Epoch 796/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3731
Epoch 797/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3610
Epoch 798/1000
20/2

20/20 [==============================] - 0s 2ms/step - loss: 0.4235
Epoch 885/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3753
Epoch 886/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3574
Epoch 887/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3962
Epoch 888/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3946
Epoch 889/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3842
Epoch 890/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4094
Epoch 891/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3620
Epoch 892/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3649
Epoch 893/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3691
Epoch 894/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4329
Epoch 895/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3704
Epoch 896/1000
20/2

20/20 [==============================] - 0s 2ms/step - loss: 0.3836
Epoch 983/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4217
Epoch 984/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.3758
Epoch 985/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3780
Epoch 986/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3750
Epoch 987/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3641
Epoch 988/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3663
Epoch 989/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4477
Epoch 990/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3633
Epoch 991/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3467
Epoch 992/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.3875
Epoch 993/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.4576
Epoch 994/1000
20/2

Predict the results

In [5]:
predictions_class = heart_disease_model.predict_classes(normalized_features_test).swapaxes(0, 1).flatten()

/Users/mhogan/.conda/envs/heartfailurepred/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Calculate intermediate results

In [6]:
true_classes = enums.HeartDiseaseClassification()
true_positive = np.count_nonzero(np.logical_and(predictions_class == true_classes.HeartDisease,
                                                labels_test == true_classes.HeartDisease).astype(int))
false_positive = np.count_nonzero(np.logical_and(predictions_class == true_classes.HeartDisease,
                                                 labels_test == true_classes.Normal).astype(int))
false_negative = np.count_nonzero(np.logical_and(predictions_class == true_classes.Normal,
                                                 labels_test == true_classes.HeartDisease).astype(int))
recall = true_positive / (false_negative + true_positive)
precision = true_positive / (true_positive + false_positive)
true_negative = np.count_nonzero(np.logical_and(predictions_class == true_classes.Normal,
                                                labels_test == true_classes.Normal).astype(int))
positives = np.count_nonzero((labels_test == true_classes.HeartDisease).astype(int))
negatives = len(labels_test) - positives

Calculate metrics

In [7]:
sensitivity = float(true_positive) / positives
specificity = float(true_negative) / negatives
accuracy = (true_positive + true_negative) / (positives + negatives)
f1_score = 2. / (1./recall + 1./precision)
mcc = (true_positive * true_negative) - (false_positive * false_negative)
mcc /= np.sqrt((true_positive + false_positive) * (true_positive + false_negative) *
               (true_negative + false_positive) * (true_negative + false_negative))
print('accuracy = ', accuracy)
print('sensitivity = ', sensitivity)
print('specificity = ', specificity)
print('f1_score = ', f1_score)
print('Matthew\'s Corrleation Coefficient = ', mcc)


accuracy =  0.8283828382838284
sensitivity =  0.8757763975155279
specificity =  0.7746478873239436
f1_score =  0.844311377245509
Matthew's Corrleation Coefficient =  0.6557814544062806
